# Обзор
Построение финальной модели на всем датасете.

In [1]:
import sys
import numpy as np
import pandas as pd
import os
import gc
import seaborn as sns
import matplotlib.pyplot as plt
import time
import pickle
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from scipy.sparse import csr_matrix

import networkx as nx
import hier
from hier import HiClassifier
import importlib
importlib.reload(hier)

from catboost import CatBoostClassifier

pd.set_option("max_colwidth", 25)
pd.set_option("display.precision", 1)
pd.options.display.float_format = "{:.3f}".format

RANDOM_STATE = 34
DIR_MODELS = 'models/'
os.makedirs(DIR_MODELS, exist_ok = True)

Загрузим подготовленные датасеты:

In [2]:
# обработанные трейн и тест
X = pd.read_parquet('train_data.parquet')
X_test = pd.read_parquet('test_data.parquet')

# граф категорий
G = nx.read_gpickle("graph.gpickle")

# 1 попытка с catboost (не удалась)
Попробовал построить модель сложнее используя кетбуст, но он показал низкие результаты на валидационной выборке:

In [8]:
y = X['category_id'].to_numpy()

X_train, X_val, y_train, y_val = train_test_split(
    X,y,
    stratify=y,
    test_size=0.2,
    random_state=RANDOM_STATE,
)

X_train.reset_index(drop=True, inplace=True)
X_val.reset_index(drop=True, inplace=True)

X_train.shape, X_val.shape, y_train.shape

((226720, 11), (56680, 11), (226720,))

In [4]:
def non_null_rows(x):
    return np.diff(x.indptr) != 0

# удаление образцов с нулевыми строками после tfidf
def tfidf_without_null_rows(tf, X_train, y_train):
    X_train_tf = tf.fit_transform(X_train)    
    ids = non_null_rows(X_train_tf)   
    return X_train_tf[ids], y_train[ids]

In [10]:
tf = TfidfVectorizer(min_df=10, lowercase=False)

X_train_tf, y_train_tf = tfidf_without_null_rows(tf, X_train['title_desc_chars'], y_train)
X_val_tf = tf.transform(X_val['title_desc_chars'])
X_train_tf.shape

(226649, 11163)

Определяем локальный классификатор catboost, который будет обучаться и упрощенно оценивать необходимое количество итераций без кроссвалидации.

In [11]:
class LocalCatBoost:
    def __init__(self, node_id, lr=0.01):

        self.is_fitted = False
        self.path_model = f"{DIR_MODELS}model{node_id}"
        self.lr = lr

        # если модель в этом узле графа тренировалась, значит она ее можно загрузить из папки      
        if os.path.isfile(self.path_model):
            self.model = CatBoostClassifier()
            self.model.load_model(self.path_model)
            self.is_fitted = True
        else:
            self.model = CatBoostClassifier(
                iterations=7000,
                random_seed=RANDOM_STATE,
                task_type="GPU",
                early_stopping_rounds=30,
                learning_rate=lr,
                eval_metric="TotalF1",
            )

    def fit(self, X, y):

        if self.is_fitted:
            return self

        X_train, X_val, y_train, y_val = train_test_split(
            X,
            y,
            stratify=y,
            test_size=0.2,
            random_state=RANDOM_STATE,
        )

        self.model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=500)
        # обучаемся на полном локальном наборе, используя количество итераций, найденное на отложенной выборке. В этой точке лучше находить кол-во итераций по кроссвалидации или вообще усреднять по нескольким моделям, пока оставил этот простой вариант.
        self.model = CatBoostClassifier(
            iterations=self.model.get_best_iteration(),
            random_seed=RANDOM_STATE,
            task_type="GPU",
            learning_rate=self.lr,
        )
        self.model.fit(X, y)
        self.model.save_model(self.path_model)

        return self

    @property
    def classes_(self):
        return self.model.classes_

    def predict_proba(self, X):
        return self.model.predict_proba(X)

Определим функцию, которая будет на большие локальные разбиения выдавать кетбуст, а на маленькие (меньше 100 образцов) - лог регрессию, потому что на маленьких выборках могут проблемы с train_test_split и самим кетбустом.

In [ ]:
def make_local_estimator(node_id, graph):
    if graph.nodes[node_id]['X'].count_nonzero() < 100:
        return LogisticRegression(C=3,
    multi_class="multinomial", max_iter=1000, random_state=RANDOM_STATE
)
    return LocalCatBoost(node_id)

In [1]:
hiclf = HiClassifier(base_estimator=make_local_estimator,
        class_hierarchy=G)
hiclf.fit(X_train_tf,y_train_tf)
y_pred = hiclf.predict(X_val_tf)
hP, hR, hF1 = hier.h_scores(y_val, y_pred, G)

print(f"hP={hP} hR={hR} hF1={hF1} time_fit={hiclf.time_fit}min time_predict={hiclf.time_predict}min")

hP=0.876 hR=0.875 hF1=0.876 time_fit=103.1min time_predict=37.5min


Получили низкие результаты, так как возможно кетбуст с такими разреженными данными работает хуже, чем если бы ему передали текстовые данные через его нативные text_features. 

# 2 попытка only LogisticRegression
Поэтому обучаем финальную модель с помощью логрегрессии в качестве локального классификатора. Построим предсказания для X_test используя весь тренировочный набор:

In [9]:
tf = TfidfVectorizer(min_df=10, lowercase=False)

y = X['category_id'].to_numpy()
X_train_tf, y_train_tf = tfidf_without_null_rows(tf, X['title_desc_chars'], y)
X_test_tf = tf.transform(X_test['title_desc_chars'])

X_train_tf.shape, X_test_tf.shape

((283333, 12551), (70864, 12551))

In [10]:
lg = LogisticRegression(
    C=3, multi_class="multinomial", max_iter=1000, random_state=RANDOM_STATE
)

hiclf = HiClassifier(base_estimator=lg, class_hierarchy=G)
hiclf.fit(X_train_tf, y_train_tf)
y_pred = hiclf.predict(X_test_tf)
y_pred[:3]

Building features:   0%|          | 0/1476 [00:00<?, ?it/s]

Training base classifiers:   0%|          | 0/1476 [00:00<?, ?it/s]

array([11574, 11878, 13299])

In [11]:
submission = pd.DataFrame()
submission['id'] = X_test['id']
submission['predicted_category_id'] = y_pred
submission.to_parquet('result.parquet',index=False)